In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName('CommonDFandSQLOperation').getOrCreate()

In [4]:
spark.sql("SET -v").select("key", "value").show(n=5, truncate=False)

+---------------------------------------------------------+----------------------------------------------------------------+
|key                                                      |value                                                           |
+---------------------------------------------------------+----------------------------------------------------------------+
|spark.sql.adaptive.advisoryPartitionSizeInBytes          |<value of spark.sql.adaptive.shuffle.targetPostShuffleInputSize>|
|spark.sql.adaptive.coalescePartitions.enabled            |true                                                            |
|spark.sql.adaptive.coalescePartitions.initialPartitionNum|<undefined>                                                     |
|spark.sql.adaptive.coalescePartitions.minPartitionNum    |<undefined>                                                     |
|spark.sql.adaptive.enabled                               |false                                                           |


In [5]:
airportsna = (spark.read.format('csv').options(header='true', inferSchema = 'true', sep ='\t').load('airport-codes-na.txt'))

In [6]:
airportsna.createOrReplaceTempView('airports_na')

In [7]:
spark.sql('SELECT * FROM airports_na LIMIT 10').show()

+-----------+-----+-------+----+
|       City|State|Country|IATA|
+-----------+-----+-------+----+
| Abbotsford|   BC| Canada| YXX|
|   Aberdeen|   SD|    USA| ABR|
|    Abilene|   TX|    USA| ABI|
|      Akron|   OH|    USA| CAK|
|    Alamosa|   CO|    USA| ALS|
|     Albany|   GA|    USA| ABY|
|     Albany|   NY|    USA| ALB|
|Albuquerque|   NM|    USA| ABQ|
| Alexandria|   LA|    USA| AEX|
|  Allentown|   PA|    USA| ABE|
+-----------+-----+-------+----+



In [7]:
departureDelays = (spark.read.format('csv').options(header = 'true').load('departuredelays.csv'))

In [8]:
departureDelays = (departureDelays
                      .withColumn('delay',expr("CAST(delay as INT) as delay"))
                      .withColumn('distance', expr("CAST(distance as INT) as distance")))

In [9]:
departureDelays.createOrReplaceTempView("departureDelays")

In [10]:
foo = (departureDelays.filter(expr("""origin =='SEA' and destination =='SFO' and date like'01010%' and delay>0""")))

In [11]:
foo.createOrReplaceTempView('foo')

In [13]:
spark.sql('SELECT * FROM foo LIMIT 10').show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [14]:
#UNION
bar = departureDelays.union(foo)
bar.createOrReplaceTempView("bar")

In [15]:
spark.sql("""SELECT * FROM bar
            WHERE origin ='SEA' AND destination='SFO' AND date like '01010%' and delay>0""").show()

+--------+-----+--------+------+-----------+
|    date|delay|distance|origin|destination|
+--------+-----+--------+------+-----------+
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
|01010710|   31|     590|   SEA|        SFO|
|01010955|  104|     590|   SEA|        SFO|
|01010730|    5|     590|   SEA|        SFO|
+--------+-----+--------+------+-----------+



In [16]:
#Join departure delay with airport info
departureDelays.join(airportsna, airportsna.IATA == departureDelays.origin).select('City','State',"origin", "date", "delay", "distance", "destination").show(10)

+---------+-----+------+--------+-----+--------+-----------+
|     City|State|origin|    date|delay|distance|destination|
+---------+-----+------+--------+-----+--------+-----------+
|Allentown|   PA|   ABE|01011245|    6|     602|        ATL|
|Allentown|   PA|   ABE|01020600|   -8|     369|        DTW|
|Allentown|   PA|   ABE|01021245|   -2|     602|        ATL|
|Allentown|   PA|   ABE|01020605|   -4|     602|        ATL|
|Allentown|   PA|   ABE|01031245|   -4|     602|        ATL|
|Allentown|   PA|   ABE|01030605|    0|     602|        ATL|
|Allentown|   PA|   ABE|01041243|   10|     602|        ATL|
|Allentown|   PA|   ABE|01040605|   28|     602|        ATL|
|Allentown|   PA|   ABE|01051245|   88|     602|        ATL|
|Allentown|   PA|   ABE|01050605|    9|     602|        ATL|
+---------+-----+------+--------+-----+--------+-----------+
only showing top 10 rows



In [17]:
#Join foo with airports info
(foo.join(airportsna, airportsna.IATA == foo.origin)
     .select('City','State',"origin", "date", "delay", "distance", "destination").show(10))

+-------+-----+------+--------+-----+--------+-----------+
|   City|State|origin|    date|delay|distance|destination|
+-------+-----+------+--------+-----+--------+-----------+
|Seattle|   WA|   SEA|01010710|   31|     590|        SFO|
|Seattle|   WA|   SEA|01010955|  104|     590|        SFO|
|Seattle|   WA|   SEA|01010730|    5|     590|        SFO|
+-------+-----+------+--------+-----+--------+-----------+



In [18]:
departureDelaysWindow = spark.sql("""SELECT origin, destination, SUM(delay) AS TotalDelays
                FROM departureDelays 
                WHERE origin IN ('SEA', 'SFO', 'JFK')
                AND destination IN ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
                GROUP BY origin, destination""")
departureDelaysWindow.createOrReplaceTempView('departureDelaysWindow')

In [19]:
spark.sql("SELECT * FROM departureDelaysWindow").show()

+------+-----------+-----------+
|origin|destination|TotalDelays|
+------+-----------+-----------+
|   JFK|        ORD|       5608|
|   SEA|        LAX|       9359|
|   JFK|        SFO|      35619|
|   SFO|        ORD|      27412|
|   JFK|        DEN|       4315|
|   SFO|        DEN|      18688|
|   SFO|        SEA|      17080|
|   SEA|        SFO|      22293|
|   JFK|        ATL|      12141|
|   SFO|        ATL|       5091|
|   SEA|        DEN|      13645|
|   SEA|        ATL|       4535|
|   SEA|        ORD|      10041|
|   JFK|        SEA|       7856|
|   JFK|        LAX|      35755|
|   SFO|        JFK|      24100|
|   SFO|        LAX|      40798|
|   SEA|        JFK|       4667|
+------+-----------+-----------+



In [20]:
#With each of these origin airports you wanted to find the three destination that experienced the most delays
spark.sql("""SELECT origin, destination, TotalDelays, rank
            FROM (
                SELECT origin, destination, TotalDelays, dense_rank()
                    OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank FROM departureDelaysWindow) t
                WHERE rank<=3
                    """).show()

+------+-----------+-----------+----+
|origin|destination|TotalDelays|rank|
+------+-----------+-----------+----+
|   SEA|        SFO|      22293|   1|
|   SEA|        DEN|      13645|   2|
|   SEA|        ORD|      10041|   3|
|   SFO|        LAX|      40798|   1|
|   SFO|        ORD|      27412|   2|
|   SFO|        JFK|      24100|   3|
|   JFK|        LAX|      35755|   1|
|   JFK|        SFO|      35619|   2|
|   JFK|        ATL|      12141|   3|
+------+-----------+-----------+----+



In [52]:
#add new column
foo1 = (foo.withColumn("status", expr("CASE WHEN delay<=10 THEN 'On-time' ELSE 'DELAYED' END")))
foo1.show()

+--------+-----+--------+------+-----------+-------+
|    date|delay|distance|origin|destination| status|
+--------+-----+--------+------+-----------+-------+
|01010710|   31|     590|   SEA|        SFO|DELAYED|
|01010955|  104|     590|   SEA|        SFO|DELAYED|
|01010730|    5|     590|   SEA|        SFO|On-time|
+--------+-----+--------+------+-----------+-------+



In [53]:
foo2 = foo1.drop('delay')
foo2.show()

+--------+--------+------+-----------+-------+
|    date|distance|origin|destination| status|
+--------+--------+------+-----------+-------+
|01010710|     590|   SEA|        SFO|DELAYED|
|01010955|     590|   SEA|        SFO|DELAYED|
|01010730|     590|   SEA|        SFO|On-time|
+--------+--------+------+-----------+-------+



In [54]:
foo3 = foo2.withColumnRenamed('status','delay-status')
foo3.show()

+--------+--------+------+-----------+------------+
|    date|distance|origin|destination|delay-status|
+--------+--------+------+-----------+------------+
|01010710|     590|   SEA|        SFO|     DELAYED|
|01010955|     590|   SEA|        SFO|     DELAYED|
|01010730|     590|   SEA|        SFO|     On-time|
+--------+--------+------+-----------+------------+



In [58]:
spark.sql("""SELECT destination, CAST(SUBSTRING(date, 0,2) AS int) AS month, delay FROM departureDelays WHERE origin='SEA'""").show()

+-----------+-----+-----+
|destination|month|delay|
+-----------+-----+-----+
|        ORD|    1|   92|
|        JFK|    1|   -7|
|        DFW|    1|   -5|
|        MIA|    1|   -3|
|        DFW|    1|   -3|
|        DFW|    1|    1|
|        ORD|    1|  -10|
|        DFW|    1|   -6|
|        DFW|    1|   -2|
|        ORD|    1|   -3|
|        ORD|    1|    0|
|        DFW|    1|   23|
|        DFW|    1|   36|
|        ORD|    1|  298|
|        JFK|    1|    4|
|        DFW|    1|    0|
|        MIA|    1|    2|
|        DFW|    1|    0|
|        DFW|    1|    0|
|        ORD|    1|   83|
+-----------+-----+-----+
only showing top 20 rows



In [30]:
#Pivot table
spark.sql("""SELECT destination,Feb_AvgDelay, Feb_MaxDelay,IFNULL(Mar_AvgDelay,0) AS Mar_AvgDelay, IFNULL(Mar_MaxDelay,0) AS Mar_MaxDelay  
                FROM (SELECT destination, CAST(SUBSTRING(date,0,2) AS INT) AS month, delay
                FROM departureDelays WHERE origin ='SEA')
                PIVOT(
                    CAST(AVG(delay) AS DECIMAL(4,2)) AS AvgDelay, MAX(delay) AS MaxDelay
                    FOR month in (2 Feb, 3 Mar))
                ORDER BY destination""").show()

+-----------+------------+------------+------------+------------+
|destination|Feb_AvgDelay|Feb_MaxDelay|Mar_AvgDelay|Mar_MaxDelay|
+-----------+------------+------------+------------+------------+
|        ABQ|       11.42|          69|       11.47|          74|
|        ANC|        7.90|         141|        5.10|         187|
|        ATL|        7.73|         145|        6.53|         109|
|        AUS|       -0.21|          18|        4.03|          61|
|        BOS|       14.58|         152|        7.78|         119|
|        BUR|       -1.89|          78|        2.01|         108|
|        CLE|        null|        null|        0.00|           0|
|        CLT|       12.96|         228|        5.16|         110|
|        COS|       12.18|         203|        9.74|         205|
|        CVG|        null|        null|        0.00|           0|
|        DCA|        0.07|          34|        5.73|         199|
|        DEN|       12.95|         625|        7.48|         231|
|        D